In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.processors import build_processor
from multiplex_pipeline.processors.controller import ResourceBuildingController
from multiplex_pipeline.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [10]:
# load analysis configuration
settings_path = r'C:\BLCA-7_Analysis\analysis_settings_BLCA7.yaml'

settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/BLCA-7/BLCA-7_Final', analysis_name='BLCA-7_Analysis', local_analysis_dir='C:/', remote_analysis_dir='/ix1/kkedziora/blca_analysis', log_dir=None), core_detection=CoreDetectionSettings(detection_image='BLCA-7_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=['008_ECad'], use_markers=['DAPI', 'CD45', 'CD44', 'HLA1', 'NaKATPase', 'CD11C', 'pCK26', 'pS6', 'HES1'], ignore_markers=['Antibody1', 'TNFa', 'Snail1', 'SKP2', 'ProgRc', 'Plk1', 'PH3', 'PDL1', 'p65', 'p130', 'p-p130', 'p-Cdc6', 'LAG3', 'IL-8', 'HER2', 'ERa', 'EpCAM', 'E2F1', 'cycD3', 'cycB2', 'CDC25C', 'CD86', 'CD73', 'CD69', 'CD62L', 'CD56', 'CD4', 'CD25', 'CD19', 'CD27', 'CCR7', 'cCASP3'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanu

### Define the logger

In [11]:
log_file = settings.log_dir_path / f"quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [12]:
core_dir = settings.analysis_dir / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-7_Analysis/cores/Core_000.zarr')]

### Setup

In [13]:
# setup quantification controllers
quant_controller_list = [] 
for quant in settings.quant:

    table_name = quant.name
    masks_keys = quant.masks
    # # for test - delete
    # masks_keys = {k: masks_keys[k] for k in ['nucleus', 'cell'] if k in masks_keys}
    connect_to_mask = quant.layer_connection

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{connect_to_mask}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        connect_to_mask=connect_to_mask,
        overwrite=True,
    )

    quant_controller_list.append(controller) 

2025-10-17 14:30:30.302 | INFO     | __main__:<module>:11 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


### Quantify

In [16]:
for sd_path in path_list:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-10-17 14:31:12.203 | INFO     | __main__:<module>:4 - Processing Core_000.zarr


version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


2025-10-17 14:31:12.787 | INFO     | multiplex_pipeline.object_quantification.controller:validate_sdata_as_input:229 - Channels not specified. Quantifying all existing channels (9).
2025-10-17 14:31:14.207 | INFO     | multiplex_pipeline.object_quantification.controller:run:253 - Prepared masks for quantification.
2025-10-17 14:31:14.208 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:68 - Quantifying morphology features for mask 'nucleus'
2025-10-17 14:31:16.455 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:68 - Quantifying morphology features for mask 'cell'
2025-10-17 14:31:18.789 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:68 - Quantifying morphology features for mask 'ring'
2025-10-17 14:31:20.942 | INFO     | multiplex_pipeline.object_quantification.controller:build_obs:68 - Quantifying morphology features for mask 'cyto'
2025-10-17 14:31:23.032 | INFO     | multiplex_pipeline.object_quantifica

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\spatialdata\models\models.py:1144: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


2025-10-17 14:31:32.621 | INFO     | multiplex_pipeline.object_quantification.controller:run:286 - Quantification complete. Resulting AnnData has 2330 observations and 72 variables.
2025-10-17 14:31:33.326 | SUCCESS  | multiplex_pipeline.object_quantification.controller:run:313 - Quantification complete. Table 'instanseg_table' written to C:\BLCA-7_Analysis\cores\Core_000.zarr


### Sneak peek

In [17]:
sdata = sd.read_zarr(sd_path)

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [ ]:
from napari_spatialdata import Interactive

Interactive(sdata)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-17 14:31:55.576 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:31:55.577 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:09.268 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:09.275 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:15.805 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:15.810 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:23.307 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:32:23.314 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:33:09.245 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:33:09.251 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updati

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-17 14:34:35.216 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:34:35.226 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:34:35.227 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


...

2025-10-17 14:36:43.325 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:36:43.336 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:36:43.338 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-17 14:36:58.232 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:36:58.241 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:36:58.242 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-17 14:37:02.631 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:02.638 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:02.639 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:26.679 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:26.692 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:27.114 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:34.674 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:34.684 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:34.684 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-17 14:37:43.544 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updati